# Nota

He entrenado el modelo con muy pocos videos porque lo he ejecutado en local y si no eran muchisimos minutos de entrenamiento. Por eso igual el accuracy es bajo pero funcionar funcionan todos los pasos de la tarea. El video de youtube es descargado por mi. 

### Cargar Vídeos y Extraer Frames: Implementa una función para extraer frames de los vídeos

In [8]:
import cv2
import os

def extract_frames(video_path, num_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(num_frames):
        frame_index = int((i / num_frames) * total_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)

    cap.release()
    return frames


### Construir el Modelo CNN + LSTM

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed

def build_model(input_shape, num_classes):
    model = Sequential()

    # CNN para extraer características de los frames
    model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    # LSTM para capturar la dependencia temporal
    model.add(LSTM(64))

    # Capa de salida
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


### Preparación de datos y entrenamiento del modelo

In [10]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

base_path = 'UCF50'
video_paths = []
labels = []
for deporte in os.listdir(base_path):
    deporte_path = os.path.join(base_path, deporte)
    if os.path.isdir(deporte_path):
        for video in os.listdir(deporte_path):
            video_path = os.path.join(deporte_path, video)
            video_paths.append(video_path)
            labels.append(deporte)

label_mapping = {label: idx for idx, label in enumerate(set(labels))}
labels = [label_mapping[label] for label in labels]
frames = [extract_frames(path) for path in video_paths]
frames = [f for f in frames if len(f) > 0]
labels = [l for f, l in zip(frames, labels) if len(f) > 0]
X = np.array(frames)
y = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]
model = build_model(input_shape, num_classes)
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.2)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 102s 21s/step - accuracy: 0.1792 - loss: 1.8421 - val_accuracy: 0.1250 - val_loss: 1.8992
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 73s 19s/step - accuracy: 0.1875 - loss: 1.7713 - val_accuracy: 0.1250 - val_loss: 1.9320
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 47s 12s/step - accuracy: 0.1125 - loss: 1.8508 - val_accuracy: 0.1250 - val_loss: 1.9062
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 10s/step - accuracy: 0.1708 - loss: 1.7836 - val_accuracy: 0.1250 - val_loss: 1.8825
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 44s 11s/step - accuracy: 0.2792 - loss: 1.6483 - val_accuracy: 0.1250 - val_loss: 1.8033
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step - accuracy: 0.1458 - loss: 1.7514 - val_accuracy: 0.1250 - val_loss: 1.7870
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 52s 15s/step - accuracy: 0.1708 - loss: 1.6099 - val_accuracy: 0.1250 - val_loss: 1.7756
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 46s 11s/step - accuracy: 0.1792 - loss: 1.6122 - val_accuracy: 0.1250 - val_loss: 1.7650


### Función de inferencia

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


def predict_video(model, video_path, label_mapping):

    video_path = r"UCF50\Basketball\v_Basketball_g01_c07.avi"

    # Extraer frames del video
    frames = extract_frames(video_path)
    
    if len(frames) == 0:
        print(f"⚠️ No se pudieron extraer frames del video: {video_path}")
        return None

    # Convertir los frames a formato adecuado para el modelo
    frames = np.array(frames)
    frames = np.expand_dims(frames, axis=0)  # Añadir dimensión batch

    # Hacer la predicción
    prediction = model.predict(frames)
    predicted_class = np.argmax(prediction)

    # Obtener el nombre del deporte a partir del índice predicho
    class_name = {v: k for k, v in label_mapping.items()}[predicted_class]

    return class_name

# Uso de la función
video_path = "UCF50\Basketball\v_Basketball_g02_c01.avi"
predicted_sport = predict_video(model, video_path, label_mapping)
print(f"🏅 Deporte predicho: {predicted_sport}")


<>:31: SyntaxWarning: invalid escape sequence '\B'
<>:31: SyntaxWarning: invalid escape sequence '\B'
C:\Users\usuario\AppData\Local\Temp\ipykernel_16368\899247916.py:31: SyntaxWarning: invalid escape sequence '\B'
  video_path = "UCF50\Basketball\v_Basketball_g02_c01.avi"


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
🏅 Deporte predicho: Rowing


### Test Externo

In [12]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, TimeDistributed, Dense, Flatten

# 📌 FUNCIÓN PARA EXTRAER FRAMES
def extract_frames(video_path, num_frames=10, target_size=(64, 64)):
    """
    Extrae un número fijo de frames de un video.

    Parámetros:
        video_path: Ruta del video.
        num_frames: Número de frames a extraer.
        target_size: Tamaño al que se redimensionarán los frames.

    Retorna:
        Un array de frames normalizados o None si el video no se puede leer.
    """
    cap = cv2.VideoCapture(video_path)

    # Verificar si el video se abrió correctamente
    if not cap.isOpened():
        print(f"⚠️ No se pudo abrir el video: {video_path}")
        return None

    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print(f"⚠️ El video no tiene frames: {video_path}")
        cap.release()
        return None

    for i in range(num_frames):
        frame_index = int((i / num_frames) * total_frames)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, target_size)  # Redimensionar
            frame = frame / 255.0  # Normalizar
            frames.append(frame)

    cap.release()

    # Verificar si se extrajeron suficientes frames
    if len(frames) < num_frames:
        print(f"⚠️ Solo se extrajeron {len(frames)} frames en lugar de {num_frames}. Completando con frames vacíos...")
        while len(frames) < num_frames:
            frames.append(np.zeros((target_size[0], target_size[1], 3)))  # Agregar frames negros

    return np.array(frames, dtype=np.float32)


# 📌 FUNCIÓN PARA HACER LA PREDICCIÓN
def predict_video(model, video_path, label_mapping, num_frames=10, target_size=(64, 64)):
    """
    Realiza la predicción del deporte en un video.

    Parámetros:
        model: Modelo entrenado.
        video_path: Ruta del video a clasificar.
        label_mapping: Diccionario {nombre_deporte: índice}.
        num_frames: Número de frames que el modelo espera.
        target_size: Dimensión de los frames.

    Retorna:
        Nombre del deporte predicho.
    """
    # Extraer frames del video
    frames = extract_frames(video_path, num_frames=num_frames, target_size=target_size)

    # Si la extracción falló, retornar None
    if frames is None:
        print("⚠️ No se pudo procesar el video.")
        return None

    # Agregar dimensión batch y canal
    frames = np.expand_dims(frames, axis=0)  # (1, num_frames, height, width, channels)

    # Predicción
    prediction = model.predict(frames)
    predicted_class = np.argmax(prediction)

    # Obtener el nombre del deporte a partir del índice predicho
    class_name = {v: k for k, v in label_mapping.items()}[predicted_class]

    return class_name


# 📌 MODELO CNN + LSTM
def build_model(input_shape, num_classes):
    model = Sequential()

    # CNN para extraer características
    model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))

    # LSTM para capturar dependencia temporal
    model.add(LSTM(64, return_sequences=False))

    # Capa de salida
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# 📌 CARGAR EL MODELO
input_shape = (10, 64, 64, 3)  # 10 frames, tamaño 64x64, 3 canales (RGB)
num_classes = len(label_mapping)  # Número total de deportes
model = build_model(input_shape, num_classes)

# 📌 PRUEBA CON EL VIDEO DESCARGADO
video_path = "video yt/videoplayback.webm"
predicted_sport = predict_video(model, video_path, label_mapping)

# 📌 MOSTRAR RESULTADO
if predicted_sport:
    print(f"🏅 Deporte predicho: {predicted_sport}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step
🏅 Deporte predicho: YoYo
